<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/multi-threaded-downloads/multithread_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup as b4
import asyncio
from google.colab import files
import os
import requests
import time
import threading
import zipfile
!pip install selenium

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
TRIAL = 3

### Download each individual chapter

In [ ]:
def getChapter(url, browser, trial):
  try:
      browser.get(url)
      print(f"attempting to download url: {url}")
      timeout_in_seconds = 10
      WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
      html = browser.page_source
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        file.close()
        print("I give up...")
      else:
        print(f"Download failed trial {trial}")
        getChapter(url, browser, trial + 1)

### Parse the novel's mainpage

In [ ]:
def getNovelPage(url, browser, trial):
  try:
      print(f"getNovelPage trying url: {url}...")
      browser.get(url)
      html = browser.page_source
      soup = b4(html, features="html.parser")
      strainer = soup.find(class_="chapter-list").find_all('a')
      for link in strainer:
        getChapter("https://www.lightnovelpub.com"+ link['href'], browser, 0)
      return html
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        return -1
        file.close()
        print("I give up...")
      else:
        print(f"Download failed trial {trial}")
        getNovelPage(url, browser, trial + 1)
  finally:
      browser.quit()

### Get the page of all the book's chapters

In [ ]:
url = "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities/chapters"
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
browser = webdriver.Firefox(options=options)
getNovelPage(url, browser, 0)